# FLIP (01) Practical Data Science

---
Team Director: Shaoni Wang | snwang@tulip.academy<br />

TULIP Academy <br />
http://www.tulip.academy

---

## Session 64 Analyzing the Meaning of Sentences

### Natural Language Understanding

#### Querying a Database

In this section,we will show that solving the task in a restricted domain is pretty straightforward. But we will also see that to address the problem in a more general way, we have to open up a whole new box of ideas and techniques, involving the representation of meaning.

In [ ]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

In [ ]:
from nltk import load_parser

In [ ]:
cp = load_parser('grammars/book_grammars/sql0.fcfg')

In [ ]:
query = 'What cities are located in China'

In [ ]:
trees = cp.nbest_parse(query.split())

In [ ]:
answer = trees[0].node['sem']

In [ ]:
q = ' '.join(answer)

In [ ]:
print q

In [ ]:
from nltk.sem import chat80

In [ ]:
rows = chat80.sql_query('corpora/city_database/city.db', q)

In [ ]:
for r in rows:
    print r[0]

# Propositional Logic

A logical language is designed to make reasoning formally explicit. As a result, it can capture aspects of natural language which determine whether a set of sentences is consistent. As part of this approach, we need to develop logical representations of a sentence φ that formally capture the truth-conditions of φ.

In [ ]:
nltk.boolean_ops()

In [ ]:
lp = nltk.LogicParser()

In [ ]:
lp.parse('-(P & Q)')

In [ ]:
lp.parse('P & Q')

In [ ]:
lp.parse('P | (R -> Q)')

In [ ]:
lp.parse('P <-> -- P')

In [ ]:
lp = nltk.LogicParser()

In [ ]:
SnF = lp.parse('SnF')

In [ ]:
NotFnS = lp.parse('-FnS')

In [ ]:
R = lp.parse('SnF -> -FnS')

In [ ]:
prover = nltk.Prover9()

In [ ]:
prover.prove(NotFnS, [SnF, R])

In [ ]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [ ]:
val['P']

In [ ]:
dom = set([])

In [ ]:
g = nltk.Assignment(dom)

In [ ]:
m = nltk.Model(dom, val)

In [ ]:
print m.evaluate('(P & Q)', g)

In [ ]:
print m.evaluate('-(P & Q)', g)

In [ ]:
print m.evaluate('(P & R)', g)

In [ ]:
print m.evaluate('(P | R)', g)

# First-Order Logic

In the remainder of this chapter, we will represent the meaning of natural language expressions by translating them into first-order logic. Not all of natural language semantics can be expressed in first-order logic. But it is a good choice for computational semantics because it is expressive enough to represent many aspects of semantics, and on the other hand, there are excellent systems available off the shelf for carrying out automated inference in first-order logic.

## Syntax

First-order logic keeps all the Boolean operators of propositional logic, but it adds some important new mechanisms.

In [ ]:
tlp = nltk.LogicParser(type_check=True)

In [ ]:
parsed = tlp.parse('walk(angus)')

In [ ]:
parsed.argument

In [ ]:
parsed.argument.type

In [ ]:
parsed.function

In [ ]:
parsed.function.type

In [ ]:
sig = {'walk': '<e, t>'}

In [ ]:
parsed = tlp.parse('walk(angus)', sig)

In [ ]:
lp = nltk.LogicParser()

In [ ]:
lp.parse('dog(cyril)').free()

In [ ]:
lp.parse('dog(x)').free()

In [ ]:
lp.parse('own(angus, cyril)').free()

In [ ]:
lp.parse('exists x.dog(x)').free()

In [ ]:
lp.parse('((some x. walk(x)) -> sing(x))').free()

In [ ]:
lp.parse('exists x.own(y, x)').free()

## First-Order Theorem Proving

The general case in theorem proving is to determine whether a formula that we want to prove (a proof goal) can be derived by a finite sequence of inference steps from a list of assumed formulas.

In [ ]:
NotFnS = lp.parse('-north_of(f, s)')

In [ ]:
SnF = lp.parse('north_of(s, f)')

In [ ]:
R = lp.parse('all x. all y. (north_of(x, y) -> -north_of(y, x))')

In [ ]:
prover = nltk.Prover9()

In [ ]:
prover.prove(NotFnS, [SnF, R])

In [ ]:
FnS = lp.parse('north_of(f, s)')

In [ ]:
prover.prove(FnS, [SnF, R])

## Truth in Model

Relations are represented semantically in NLTK in the standard set-theoretic way: as sets of tuples. For example, let’s suppose we have a domain of discourse consisting of the individuals Bertie, Olive, and Cyril, where Bertie is a boy, Olive is a girl, and Cyril is a dog.

In [ ]:
dom = set(['b', 'o', 'c'])

In [ ]:
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """

In [ ]:
val = nltk.parse_valuation(v)

In [ ]:
print val

In [ ]:
('o', 'c') in val['see']

In [ ]:
('b',) in val['boy']

## Individual Variables and Assignments

In our models, the counterpart of a context of use is a variable assignment. This is a mapping from individual variables to entities in the domain. Assignments are created using the Assignment constructor, which also takes the model’s domain of discourse as a parameter.

In [ ]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])

In [ ]:
g

In [ ]:
print g

In [ ]:
m = nltk.Model(dom, val)

In [ ]:
m.evaluate('see(olive, y)', g)

In [ ]:
g['y']

In [ ]:
m.evaluate('see(y, x)', g)

In [ ]:
g.purge()

In [ ]:
m.evaluate('see(olive, y)', g)

In [ ]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

## Quantification

One of the crucial insights of modern logic is that the notion of variable satisfaction can be used to provide an interpretation for quantified formulas.

In [ ]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

In [ ]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

In [ ]:
fmla1 = lp.parse('girl(x) | boy(x)')

In [ ]:
m.satisfiers(fmla1, 'x', g)

In [ ]:
fmla2 = lp.parse('girl(x) -> walk(x)')

In [ ]:
m.satisfiers(fmla2, 'x', g)

In [ ]:
fmla3 = lp.parse('walk(x) -> girl(x)')

In [ ]:
m.satisfiers(fmla3, 'x', g)

In [ ]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

## Quantifier Scope Ambiguity

In [ ]:
v2 = """
    bruce => b
    cyril => c
    elspeth => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m), (c, a)}
    """

In [ ]:
dom2 = val2.domain

In [ ]:
m2 = nltk.Model(dom2, val2)

In [ ]:
g2 = nltk.Assignment(dom2)

In [ ]:
fmla4 = lp.parse('(person(x) -> exists y.(person(y) & admire(x, y)))')

In [ ]:
m2.satisfiers(fmla4, 'x', g2)

In [ ]:
fmla5 = lp.parse('(person(y) & all x.(person(x) -> admire(x, y)))')

In [ ]:
m2.satisfiers(fmla5, 'y', g2)

In [ ]:
fmla6 = lp.parse('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')

In [ ]:
m2.satisfiers(fmla6, 'y', g2)

## Model Building

We have been assuming that we already had a model, and wanted to check the truth of a sentence in the model. By contrast, model building tries to create a new model,given some set of sentences. If it succeeds, then we know that the set is consistent, since we have an existence proof of the model.

In [ ]:
a3 = lp.parse('exists x.(man(x) & walks(x))')

In [ ]:
c1 = lp.parse('mortal(socrates)')

In [ ]:
c2 = lp.parse('-mortal(socrates)')

In [ ]:
mb = nltk.Mace(5)

In [ ]:
print mb.build_model(None, [a3, c1])

In [ ]:
print mb.build_model(None, [a3, c2])

In [ ]:
print mb.build_model(None, [c1, c2])

In [ ]:
a4 = lp.parse('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')

In [ ]:
a5 = lp.parse('man(adam)')

In [ ]:
a6 = lp.parse('woman(eve)')

In [ ]:
g = lp.parse('love(adam,eve)')

In [ ]:
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])

In [ ]:
mc.build_model()

In [ ]:
print mc.valuation

In [ ]:
a7 = lp.parse('all x. (man(x) -> -woman(x))')

In [ ]:
g = lp.parse('love(adam,eve)')

In [ ]:
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])

In [ ]:
mc.build_model()

In [ ]:
print mc.valuation

# The Semantics of English Sentences

## The λ-Calculus

Remember that \ is a special character in Python strings. We must either escape it (with another \), or else use “raw strings”  as shown here:

In [ ]:
lp = nltk.LogicParser()

In [ ]:
e = lp.parse(r'\x.(walk(x) & chew_gum(x))')

In [ ]:
e

In [ ]:
e.free()

In [ ]:
print lp.parse(r'\x.(walk(x) & chew_gum(y))')

In [ ]:
e = lp.parse(r'\x.(walk(x) & chew_gum(x))(gerald)')

In [ ]:
print e

In [ ]:
print e.simplify()

In [ ]:
print lp.parse(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify()

In [ ]:
print lp.parse(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify()

In [ ]:
e1 = lp.parse('exists x.P(x)')

In [ ]:
print e1

In [ ]:
e2 = e1.alpha_convert(nltk.Variable('z'))

In [ ]:
print e2

In [ ]:
e1 == e2

In [ ]:
e3 = lp.parse('\P.exists x.P(x)(\y.see(y, x))')

In [ ]:
print e3

In [ ]:
print e3.simplify()

## Quantified NPs

At the start of this section, we briefly described how to build a semantic representation for Cyril barks. You would be forgiven for thinking this was all too easy—surely there is a bit more to building compositional semantics.

In [ ]:
lp = nltk.LogicParser()

In [ ]:
tvp = lp.parse(r'\X x.X(\y.chase(x,y))')

In [ ]:
np = lp.parse(r'(\P.exists x.(dog(x) & P(x)))')

In [ ]:
vp = nltk.ApplicationExpression(tvp, np)

In [ ]:
print vp

In [ ]:
print vp.simplify()

In [ ]:
from nltk import load_parser

In [ ]:
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)

In [ ]:
sentence = 'Angus gives a bone to every dog'

In [ ]:
tokens = sentence.split()

In [ ]:
trees = parser.nbest_parse(tokens)

In [ ]:
for tree in trees:
    print tree.node['SEM']

In [ ]:
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """

In [ ]:
val = nltk.parse_valuation(v)

In [ ]:
g = nltk.Assignment(val.domain)

In [ ]:
m = nltk.Model(val.domain, val)

In [ ]:
sent = 'Cyril sees every boy'

In [ ]:
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'

In [ ]:
results = nltk.batch_evaluate([sent], grammar_file, m, g)[0]

In [ ]:
for (syntree, semrel, value) in results:
    print semrep
    print value

## Quantifier Ambiguity Revisited

One important limitation of the methods described earlier is that they do not deal with scope ambiguity.

In [ ]:
from nltk.sem import cooper_storage as cs

In [ ]:
sentence = 'every girl chases a dog'

In [ ]:
trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')

In [ ]:
semrep = trees[0].node['sem']

In [ ]:
cs_semrep = cs.CooperStore(semrep)

In [ ]:
print cs_semrep.core

In [ ]:
for bo in cs_semrep.store:
    print bo

In [ ]:
cs_semrep.s_retrieve(trace=True)

In [ ]:
for reading in cs_semrep.readings:
    print reading

# Discourse Semantics

A discourse is a sequence of sentences. Very often, the interpretation of a sentence in a discourse depends on what preceded it. A clear example of this comes from anaphoric pronouns, such as he, she, and it. Given a discourse such as Angus used to have a dog.
But he recently disappeared., you will probably interpret he as referring to Angus’s dog. However, in Angus used to have a dog. He took him for walks in New Town., you are more likely to interpret he as referring to Angus himself.

## Discourse Representation Theory

In [ ]:
dp = nltk.DrtParser()

In [ ]:
drs1 = dp.parse('([x, y], [angus(x), dog(y), own(x, y)])')

In [ ]:
print drs1

In [ ]:
drs1.draw()

In [ ]:
print drs1.fol()

In [ ]:
drs2 = dp.parse('([x], [walk(x)]) + ([y], [run(y)])')

In [ ]:
print drs2

In [ ]:
print drs2.simplify()

In [ ]:
drs3 = dp.parse('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')

In [ ]:
print drs3.fol()

In [ ]:
drs4 = dp.parse('([x, y], [angus(x), dog(y), own(x, y)])')

In [ ]:
drs5 = dp.parse('([u, z], [PRO(u), irene(z), bite(u, z)])')

In [ ]:
drs6 = drs4 + drs5

In [ ]:
print drs6.simplify()

In [ ]:
print drs6.simplify().resolve_anaphora()

In [ ]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.DrtParser())

In [ ]:
trees = parser.nbest_parse('Angus owns a dog'.split())

In [ ]:
print trees[0].node['sem'].simplify()

## Discourse Processing

When we interpret a sentence, we use a rich context for interpretation, determined in part by the preceding context and in part by our background assumptions.

In [ ]:
dt = nltk.DiscourseTester(['A student dances', 'Every student is a person'])

In [ ]:
dt.readings()

In [ ]:
dt.add_sentence('No person dances', consistchk=True)

In [ ]:
dt.retract_sentence('No person dances', verbose=True)

In [ ]:
dt.add_sentence('A person dances', informchk=True)

In [ ]:
from nltk.tag import RegexpTagger
tagger = RegexpTagger(
                    [('^(chases|runs)$', 'VB'),
                    ('^(a)$', 'ex_quant'),
                    ('^(every)$', 'univ_quant'),
                    ('^(dog|boy)$', 'NN'),
                    ('^(He)$', 'PRP')
                    ])

In [ ]:
rc = nltk.DrtGlueReadingCommand(depparser=nltk.MaltParser(tagger=tagger))

In [ ]:
dt = nltk.DiscourseTester(['Every dog chases a boy', 'He runs'], rc)

In [ ]:
dt.readings()

In [ ]:
dt.readings(show_thread_readings=True)

In [ ]:
dt.readings(show_thread_readings=True, filter=True)